### API ПриватБанк ver 0.0.1

In [1]:
import requests
from datetime import datetime
from pandas import date_range
from string import punctuation

In [2]:
def remove_puctuation(x):
    return x.translate(str.maketrans('', '', punctuation))


def input_and_check(inp_mess,
                    options,
                    err_mess='Некоректний ввід',
                    date=False):
    inp = input(inp_mess).upper().split()
    if not date:
        inp = tuple(map(remove_puctuation, inp))
    if set(inp) < options and inp:
        return inp
    else:
        print(err_mess)
        again = input('Спробуєте ще раз? ')
        if again.lower() == 'так':
            return input_and_check(inp_mess, options, err_mess, date)

In [3]:
def say_hello():
    print('Вітаю! Мене звати Аполінарія Валютченко. Що я можу:',
          '1. Повідомити поточні готівкові і безготівкові курси кількох валют ПриватБанку',
          '2. Повідомити готівкові курси кількох валют НБУ за минулі дні, починаючи з 03.11.2014',
          sep='\n')
    task = input_and_check(inp_mess='Оберіть, який пункт Вас цікавить (1 або 2): ',
                           options={'1', '2'},
                           err_mess='Немає такого пункту')
    return task[0]

In [4]:
def responce(url, dict_keys, nested=False):
    resp = requests.get(url)
    match resp.status_code:
        case 200:
            print('Дані отримані')
            data = resp.json()
            if nested:
                data = data[dict_keys['nested']]
            currencies = set(map(lambda x: x[dict_keys['curr']], data))
            print(f'Доступні валюти: {currencies}')
            curr = input_and_check(inp_mess='Оберіть валюту/валюти з доступних, або всі (введіть "всі"): ',
                                   options=currencies | {"ВСІ"})
            if curr == ("ВСІ",):
                curr_data = data
            else:
                curr_data = tuple(filter(lambda x: x[dict_keys['curr']] in curr, data))
            for obj in curr_data:
                print(f'Продаж: 1 {obj[dict_keys["curr"]]} = {obj[dict_keys["buy"]]} {obj[dict_keys["base_curr"]]}',
                      f'Купівля: 1 {obj[dict_keys["curr"]]} = {obj[dict_keys["sale"]]} {obj[dict_keys["base_curr"]]}',
                      sep='\n')
        case 404:
            print('Покликання неправильне, спробуйте інше')
        case _:
            print('Щось пішло не так, спробуйте пізніше')

In [5]:
def task_1():
    url_cash = 'https://api.privatbank.ua/p24api/pubinfo?json&exchange&coursid=5'
    url_noncash = 'https://api.privatbank.ua/p24api/pubinfo?exchange&json&coursid=11'
    url_dict = {'1': url_cash, '2': url_noncash}
    dict_keys = {'curr': 'ccy', 'base_curr': 'base_ccy', 'buy': 'buy', 'sale': 'sale'}
    option = input_and_check(inp_mess='Вас цікавить готівковий (1) чи безготівковий курс (2)?: ',
                             options={'1', '2'})[0]
    if option is None:
        return None
    url = url_dict[option]
    responce(url, dict_keys)

In [6]:
def task_2():
    url = 'https://api.privatbank.ua/p24api/exchange_rates?date='
    dict_keys = {'curr': "currency", 'base_curr': "baseCurrency",
                 'buy': "purchaseRateNB", 'sale': "saleRateNB",
                 'nested': "exchangeRate"}
    start = '03.11.2014'
    end = datetime.today().date()
    val_dates = date_range(start, end, inclusive='left').strftime('%d.%m.%Y')
    date = input_and_check(inp_mess='Введіть дату у форматі "дд.мм.рррр": ',
                           options=set(val_dates),
                           err_mess='Така дата недоступна, або неправильний формат',
                           date=True)
    # на жаль всі дати не можна розглянути, видає помилку 500,
    # тому розглядаємо лише першу в списку
    if date:
        print(date[0])
        url += date[0]
        responce(url, dict_keys, nested=True)

In [7]:
def pb24():
    task = say_hello()
    dict_task = {'1': task_1, '2': task_2}
    dict_task[task]()

In [8]:
pb24()

Вітаю! Мене звати Аполінарія Валютченко. Що я можу:
1. Повідомити поточні готівкові і безготівкові курси кількох валют ПриватБанку
2. Повідомити готівкові курси кількох валют НБУ за минулі дні, починаючи з 03.11.2014
Оберіть, який пункт Вас цікавить (1 або 2): 1
Вас цікавить готівковий (1) чи безготівковий курс (2)?: 1
Дані отримані
Доступні валюти: {'USD', 'EUR'}
Оберіть валюту/валюти з доступних, або всі (введіть "всі"): всі
Продаж: 1 EUR = 40.75000 UAH
Купівля: 1 EUR = 41.75000 UAH
Продаж: 1 USD = 39.30000 UAH
Купівля: 1 USD = 39.80000 UAH
